In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import os
from datetime import datetime
from sklearn import preprocessing

In [2]:
ecg_start_time = pd.read_csv("F:\\360MoveData\\Users\\admin\\Desktop\\ECG and Apple watch HR for DTW\\Raw\\skintonestudyTIMES.csv")

In [3]:
Subject_ID = []
for i in range(1,10):
    Subject_ID.append('19-00' + str(i) + '\\')
for i in range(10,57):
    Subject_ID.append('19-0' + str(i) + '\\')
    
[Subject_ID.remove(id) for id in ['19-001\\','19-002\\','19-003\\','19-025\\','19-016\\','19-017\\','19-026\\','19-039\\','19-040\\','19-042\\','19-045\\','19-046\\','19-055\\','19-056\\','19-019\\','19-050\\','19-049\\','19-051\\']]
# remove some subjects due to the bad quality

len(Subject_ID)

38

In [4]:
file_type = ['HR.csv','Apple Watch.csv','HRT.csv']
path = 'F:\\360MoveData\\Users\\admin\\Desktop\\ECG and Apple watch HR for DTW\\Raw\\'
# Create folders for subjects
# for i in Subject_ID:
#     try:
#         os.mkdir(path + i)
#     except OSError:
#         print ("Creation of the directory %s failed" % i)

#### Downloaded from Skin Tone Equity Subjects folder

In [5]:
for i in Subject_ID:
    if i == '19-028\\' or i == '19-051\\' or i == '19-052\\':
        offset = 200
    else:
        offset = 60
    
    ecg_path = path + i + file_type[0]
    aw_path = path + i + file_type[1]
    ecg_time_path = path + i + file_type[2]

    ecg = pd.read_csv(ecg_path)
    aw = pd.read_csv(aw_path)
    ecg_time = pd.read_csv(ecg_time_path)

    ecg_ts = ecg_start_time.loc[(ecg_start_time['Subject ID']== i.replace("\\","") )]['Baseline Start 1'].values[0].split()[0].strip() + str(' 00:00:00')
    date_time_obj = datetime.strptime(ecg_ts, '%m/%d/%Y %H:%M:%S')
    ecg_ts = datetime.timestamp(date_time_obj)

    # https://www.epochconverter.com/ 
    # Remember change GMT to local time
    date_time_obj = datetime.strptime(aw.columns[1].strip(), '%Y-%m-%d %H:%M:%S')
    aw_ts = datetime.timestamp(date_time_obj)
    # time stamp start at 00:00:00 am at that day.
    ecg['time'] = ecg_time.values + ecg_ts

    # Cut the first 6 rows because they contain ecperiment information: subject_id, start time, etc.
    aw = aw.drop(range(6))

    temp = []
    for j in aw['Workout date'].values:
        temp.append(float(j) + aw_ts)
    aw['time'] = temp

    aw = aw.drop(columns = ['Workout date'])
    ecg.columns = ['HR','time']
    aw.columns = ['HR','time']

    # Cut time
    ts = max(float(aw[:1]['time'].values), float(ecg[:1]['time'].values)) 
    ts = ts + offset
    # Manually set start time as 1 minute after the beginning of the procedure to avoid the instablity.
    te = min(float(aw[-1:]['time'].values), float(ecg[-1:]['time'].values))

    # Cut the data
    hr_ECG = ecg.loc[(ecg['time'] >= ts) & (ecg['time'] <= te)]['HR'].values
    hr_aw = aw.loc[(aw['time'] >= ts) & (aw['time'] <= te)]['HR'].values
    t_ECG = ecg.loc[(ecg['time'] >= ts) & (ecg['time'] <= te)]['time'].values
    t_aw = aw.loc[(aw['time'] >= ts) & (aw['time'] <= te)]['time'].values

    # Set the start time to 0.
    t_ECG = t_ECG - min(t_ECG)
    t_aw = t_aw - min(t_aw)

    # convert string to float
    hr_AW = [] 
    for k in hr_aw:
        hr_AW.append(float(k))

    # Gaussian smooth
    hr_ECG_smooth = gaussian_filter(hr_ECG, sigma=15)
    hr_AW_smooth = gaussian_filter(hr_AW, sigma=3)

    # Normalize the heart rate data to mean = 0, std = 1.
    hr_ECG_norm = preprocessing.scale(hr_ECG_smooth)
    hr_AW_norm = preprocessing.scale(hr_AW_smooth)

    # Save the data to *.csv files
    ECGdata = {'t': t_ECG, 
    'q': hr_ECG_norm 
    }
    ecg_df = pd.DataFrame(ECGdata)

    PPGdata = {'t': t_aw, 
    'q': hr_AW_norm
    }
    ppg_df = pd.DataFrame(PPGdata)
    
    plt.plot(t_ECG,hr_ECG_norm,color='black')
    plt.plot(t_aw,hr_AW_norm,color='blue')
    plt.title(i.replace("\\","")) 
    plt.savefig(path+'figures\\'+i.replace("\\",""))
    plt.close()
    
    print('ok' + i)
    ecg_df.to_csv(path.replace('Raw\\','Preprocessed\\') + i.replace('\\','') + 'ECG.csv')
    ppg_df.to_csv(path.replace('Raw\\','Preprocessed\\') + i.replace('\\','') + 'AW.csv')

ok19-004\
ok19-005\
ok19-006\
ok19-007\
ok19-008\
ok19-009\
ok19-010\
ok19-011\
ok19-012\
ok19-013\
ok19-014\
ok19-015\
ok19-018\
ok19-020\
ok19-021\
ok19-022\
ok19-023\
ok19-024\
ok19-027\
ok19-028\
ok19-029\
ok19-030\
ok19-031\
ok19-032\
ok19-033\
ok19-034\
ok19-035\
ok19-036\
ok19-037\
ok19-038\
ok19-041\
ok19-043\
ok19-044\
ok19-047\
ok19-048\
ok19-052\
ok19-053\
ok19-054\


1. No gound truth of alignment in real life signals.(a figure showing edtw outperforms others?)
2. Waiting for the data
3. Wrap it as a opensource tool.
4 Aoi for 1:1